In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import unidecode
import warnings
warnings.filterwarnings("ignore")

In [17]:
# Cargar datasets preprocesados
df_completo = pd.read_parquet(r'..\Datasets\dataset_completo.parquet')
df_actor = pd.read_parquet(r'..\Datasets\actor_dataset.parquet')
df_director = pd.read_parquet(r'..\Datasets\director_dataset.parquet')

In [18]:
# Función para normalizar el título antes de buscarlo en el dataset (incluso si hay diferencias en mayúsculas o acentos)
def limpiar_texto_completo(texto):
    if pd.isnull(texto):
        return None
    try:
        texto = str(texto).lower().strip()
        texto = ' '.join(texto.split())  # Eliminar espacios extra
        texto = unidecode.unidecode(texto)  # Eliminar caracteres especiales
    except Exception as e:
        print(f"Error procesando el texto: {texto}, Error: {e}")
        return None 
    return texto

# Aplicar limpieza de texto en nombres de actores y directores
df_actor['actor_name'] = df_actor['actor_name'].apply(limpiar_texto_completo)
df_director['director_name'] = df_director['director_name'].apply(limpiar_texto_completo)

# Crear la matriz de características basada en TF-IDF o características numéricas
# Suponiendo que ya tienes el TF-IDF entrenado
features_matrix = ...  # Esto debe generarse a partir de tu TF-IDF o características numéricas normalizadas

In [19]:
# Función de recomendación
def recomendacion(titulo, n_recomendaciones=5):
    try:
        # Limpiar y normalizar el título de la película
        titulo_normalizado = limpiar_texto_completo(titulo)
        
        # Verificar si el título existe en el dataset
        pelicula_idx = df_completo[df_completo['title'].str.contains(titulo_normalizado, case=False, na=False)].index
        
        if pelicula_idx.empty:
            return {"mensaje": f"El título '{titulo}' no se encuentra en el dataset."}
        
        # Obtener el índice de la película en el dataset
        idx = pelicula_idx[0]
        
        # Calcular las similitudes utilizando la matriz de características
        cosine_similarities = cosine_similarity([features_matrix[idx]], features_matrix).flatten()
        
        # Obtener los índices de las películas más similares y evitar repetir la misma película
        similar_indices = cosine_similarities.argsort()[::-1][1:n_recomendaciones + 1]
        
        # Obtener los títulos de las películas recomendadas
        recomendaciones = df_completo['title'].iloc[similar_indices].tolist()
        
        return {"recomendaciones": recomendaciones}
    
    except Exception as e:
        return {"mensaje": f"Error interno en la recomendación: {str(e)}"}

In [20]:
from sklearn.preprocessing import MinMaxScaler

# Selecciona las columnas relevantes para la recomendación, como `vote_average` y `vote_count`
features = df_completo[['vote_average', 'vote_count']]

# Normaliza las columnas seleccionadas
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Guardar la matriz de características para su uso en la recomendación
features_matrix = features_scaled

# Guardar la matriz de características en un archivo .npy
np.save(r'..\Datasets\features_matrix.npy', features_matrix)

In [21]:
# Guardar el dataset en formato Parquet
df_completo.to_parquet(r'..\Datasets\dataset_completo.parquet')